# LangGraph - A simple Agent

<a target="_blank" href="https://githubtocolab.com/IT-HUSET/ai-agenter-2025/blob/main/exercises/langgraph/1.6-langgraph-agent-example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a><br/>

This notebook demonstrates building simple **Agents** using LangGraph.

![Let's build an agent](https://github.com/IT-HUSET/ai-workshop-250121/blob/main/images/llm-apps-2024.png?raw=true)

## Setup

### Install dependencies

In [ ]:
%pip install openai~=2.0 httpx~=0.28.1 --upgrade --quiet
%pip install python-dotenv~=1.0 --upgrade --quiet
%pip install python-dotenv~=1.0 docarray~=0.41.0 pypdf~=6.1 --upgrade --quiet
%pip install chromadb~=1.1.1 lark~=1.3 --upgrade --quiet
%pip install langchain~=0.3 langchain_openai~=0.3 langchain_community~=0.3.31 langchain-chroma~=0.2.6 --upgrade --quiet
%pip install langgraph~=0.6 --upgrade --quiet

# If running locally, you can do this instead:
#%uv sync

### Load environment variables

In [ ]:
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    # Get API key from Colab secrets
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    # Load from .env file for local development
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")

# Verify API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar")
        print("   → Add a secret named 'OPENAI_API_KEY'")
        print("   → Toggle 'Notebook access' to enable it")
    else:
        print("   → Create a .env file with: OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key configured!")

### Setup Chat Model

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

## Let's try some tool calling and build an _actual_ **agent**!

![Router](https://github.com/IT-HUSET/ai-workshop-250121/blob/main/images/tool-calling.png?raw=true)

### We begin by defining our "tools"
Tools can be anything from internal / external APIs, logic within the app, databases lookups, etc.

In [ ]:
from typing import Literal
from langchain_core.tools import tool

@tool
def iceland_vacation_suggestion(topic: Literal['cafes', 'volcanoes', 'activities', 'other']) -> str:
    """Suggest a vacation spot in Iceland based on the topic. If the user doesn't state a topic, use the topic 'other'.

    Args:
        topic: The topic of interest. Must be one of 'cafes', 'volcanoes', 'activities', or 'other'.
    """
    print(f"--- iceland_vacation_suggestion called with {topic} ---")

    if topic == "cafes":
        return "Kaffibarinn"
    elif topic == "volcanoes":
        return "Fagradalsfjall"
    elif topic == "activities":
        return "Inside the Volcano"
    else:
        return "Harpa"

def iceland_vacation_spot_to_avoid(topic: Literal['cafes', 'volcanoes', 'activities', 'other']) -> str:
    """Suggest a vacation spot to avoid in Iceland, based on the topic. If the user doesn't state a topic, use the topic 'other'. 'other-.

    Args:
        topic: The topic of interest. Must be one of 'cafes', 'volcanoes', 'activities', or 'other'.
    """
    print(f"iceland_vacation_spots_to_avoid called with {topic}")

    if topic == "cafes":
        return "Cafe Babalu"
    elif topic == "volcanoes":
        return "Sundhnúkagígar / Grindavík"
    elif topic == "activities":
        return "Blue Lagoon"
    else:
        return "Aluminium smelters"

### Next, we need to let the LLM know about our tools

Some things to note:
1. We bind the tools to the LLM, that is to say, we define the schema our tools and pass it to the LLM so it knows how to call them. The function `bind_tools` is a helper method that turns a list of functions into a **[JSON schema](http://json-schema.org)** that the LLM can understand.
2. We set `parallel_tool_calls=False` to ensure that the tools are called sequentially. This is important when the tools have side effects or need to be called in a specific order. And in this case, it make the example a bit clearer.

In [ ]:
tools = [iceland_vacation_suggestion, iceland_vacation_spot_to_avoid]
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

### Now, we define our "assistant" node

This time, we'll simply use a simple function to define our node.
Note, that this time, we use the predefined **`MessagesState`** instead of defining our own state object. MessageState is a simple state object with a single key, `messages`, which is a list of `AnyMessage` (base class to all message types) objects.

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with tourist information assistance about Iceland.")

# Node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

### We can now build our graph

Two things to note below:
1. We use the predefined **`ToolNode`** for our tool calling node. This takes care of executing the actual tool/function based upon information in the LLM response about a tool call.
2. We use the predefined **`tools_condition`** for our conditional edge. This will route the control flow to the tool calling node if the LLM returns information about a tool call in its response.


In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
# NOTE: Here we use the predefined tools_condition for our conditional edge
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

### Let's test it out!

In [ ]:
messages = [HumanMessage(content="Hi! I'd like do a cool activity in Iceland!")]
#messages = [HumanMessage(content="Hi! I'm visiting Iceland next year and would like to do something fun and visit a volcano!")]
#messages = [HumanMessage(content="Can you suggest a good café I should go to when I visit Iceland? And is there any place I should avoid?")]
messages = react_graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()